In [1]:
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage


In [2]:
from typing import TypedDict, Optional, List, Dict, Any

class EmailState(TypedDict):
    email: Dict[str, Any]
    is_spam: Optional[bool]
    draft_response: Optional[str]
    messages: List[Dict[str, Any]]


In [3]:
def check_spam(state: EmailState) -> EmailState:
    subject = state["email"]["subject"]
    state["is_spam"] = "oferta" in subject.lower()
    return state


In [4]:
builder = StateGraph(EmailState)
builder.add_node("check_spam", check_spam)
builder.set_entry_point("check_spam")
builder.set_finish_point("check_spam")  # o END si hay más nodos
graph = builder.compile()


In [5]:
initial_state = {
    "email": {"subject": "¡Gran oferta!", "body": "Compra ahora"},
    "messages": []
}
result = graph.invoke(initial_state)
print(result)


{'email': {'subject': '¡Gran oferta!', 'body': 'Compra ahora'}, 'is_spam': True, 'messages': []}


In [ ]:
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

# Paso 1: Configurar el modelo
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key="tu_clave_aquí")

# Paso 2: Nodo de razonamiento (Chain of Thought)
def razonamiento_node(state):
    mensaje_usuario = state["ticket"]
    prompt = [
        SystemMessage(content="Eres un asistente de soporte técnico que razona paso a paso."),
        HumanMessage(content=f"Analiza el siguiente ticket y explica paso a paso qué tipo de problema es:\n\n{mensaje_usuario}")
    ]
    razonamiento = llm(prompt).content
    return {"razonamiento": razonamiento, "ticket": mensaje_usuario}

# Paso 3: Nodo de clasificación
def clasificacion_node(state):
    razonamiento = state["razonamiento"]
    prompt = [
        SystemMessage(content="Eres un clasificador de tickets."),
        HumanMessage(content=f"Basado en el siguiente razonamiento, clasifica el ticket como 'problema técnico', 'facturación' u 'otro':\n\n{razonamiento}")
    ]
    categoria = llm(prompt).content
    return {"categoria": categoria, "razonamiento": razonamiento}

# Paso 4: Construir el grafo
builder = StateGraph()
builder.add_node("razonamiento", razonamiento_node)
builder.add_node("clasificacion", clasificacion_node)

builder.set_entry_point("razonamiento")
builder.add_edge("razonamiento", "clasificacion")
builder.add_edge("clasificacion", END)

graph = builder.compile()

# Paso 5: Ejecutar el flujo
entrada = {"ticket": "No puedo acceder a mi cuenta desde ayer. Me dice que mi contraseña es incorrecta."}
resultado = graph.invoke(entrada)

print("🧠 Razonamiento:\n", resultado["razonamiento"])
print("📂 Categoría asignada:\n", resultado["categoria"])


C:\Users\cmoncada\AppData\Local\Temp\ipykernel_26892\4130173885.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4", temperature=0)


ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 0, 'model...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

## Siguiente codigo

In [ ]:
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from datetime import datetime
import random

# Inicializar modelo
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key="TU_API_KEY")

# Estado inicial
initial_state = {
    "ticket": "No puedo acceder a mi cuenta desde ayer. Me dice que mi contraseña es incorrecta.",
    "start_time": datetime.now(),
    "true_category": "problema técnico",  # Para evaluar accuracy
}

 Nodo de Razonamiento Inicial

In [ ]:
def razonamiento_node(state):
    prompt = [
        SystemMessage(content="Eres un asistente de soporte que razona paso a paso."),
        HumanMessage(content=f"Analiza este ticket y explica paso a paso:\n\n{state['ticket']}")
    ]
    razonamiento = llm(prompt).content
    return {**state, "razonamiento": razonamiento}


Nodo de Clasificación de Ticket

In [ ]:
def clasificacion_node(state):
    prompt = [
        SystemMessage(content="Clasifica el ticket como 'problema técnico', 'facturación' u 'otro'."),
        HumanMessage(content=f"Basado en este razonamiento:\n\n{state['razonamiento']}")
    ]
    categoria = llm(prompt).content.strip().lower()
    return {**state, "predicted_category": categoria}


Nodo de Escalamiento a Humanos

In [ ]:
def escalamiento_node(state):
    # Simulación: si el ticket contiene "no puedo" o "error", escalar
    escalar = any(palabra in state["ticket"].lower() for palabra in ["no puedo", "error", "bloqueado"])
    return {**state, "escalado": escalar}


Nodo de Registro de Métricas

In [ ]:
def metricas_node(state):
    end_time = datetime.now()
    duracion = (end_time - state["start_time"]).total_seconds()

    # Accuracy
    accuracy = 1 if state["predicted_category"] == state["true_category"] else 0

    # Tasa de escalamiento (1 si escalado, 0 si no)
    escalamiento = 1 if state.get("escalado") else 0

    return {
        **state,
        "tiempo_resolucion": duracion,
        "accuracy_clasificacion": accuracy,
        "tasa_escalamiento": escalamiento
    }


Nodo de Generación de Respuesta

In [ ]:
def respuesta_node(state):
    prompt = [
        SystemMessage(content="Genera una respuesta clara y útil para el cliente."),
        HumanMessage(content=f"Ticket: {state['ticket']}\nCategoría: {state['predicted_category']}")
    ]
    respuesta = llm(prompt).content
    return {**state, "respuesta": respuesta}


Construcción del Grafo LangGraph

In [ ]:
builder = StateGraph()
builder.add_node("razonamiento", razonamiento_node)
builder.add_node("clasificacion", clasificacion_node)
builder.add_node("escalamiento", escalamiento_node)
builder.add_node("metricas", metricas_node)
builder.add_node("respuesta", respuesta_node)

builder.set_entry_point("razonamiento")
builder.add_edge("razonamiento", "clasificacion")
builder.add_edge("clasificacion", "escalamiento")
builder.add_edge("escalamiento", "metricas")
builder.add_edge("metricas", "respuesta")
builder.add_edge("respuesta", END)

graph = builder.compile()
resultado = graph.invoke(initial_state)

# Mostrar resultados
for clave in ["razonamiento", "predicted_category", "escalado", "tiempo_resolucion", "accuracy_clasificacion", "tasa_escalamiento", "respuesta"]:
    print(f"{clave.upper()}:\n{resultado[clave]}\n")
